# 进入mmpose主目录

In [1]:
import os
os.chdir('mmpose')

# 训练

In [3]:
%pip install -U albumentations
!python tools/train.py data/rtmpose-s-ear.py

Note: you may need to restart the kernel to use updated packages.
06/04 23:43:57 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr 20 2023, 19:02:41) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 21
    GPU 0: NVIDIA GeForce RTX 3060
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: gcc (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0
    PyTorch: 2.0.1
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencod

# 在测试集上评估模型精度

In [4]:
!python tools/test.py data/rtmpose-s-ear.py \
                      work_dirs/rtmpose-s-ear/epoch_20.pth

06/04 23:52:32 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr 20 2023, 19:02:41) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 21
    GPU 0: NVIDIA GeForce RTX 3060
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: gcc (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0
    PyTorch: 2.0.1
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-

# 导入工具包

In [5]:
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['axes.unicode_minus']=False  # 用来正常显示负号

# 载入训练日志

In [14]:
log_path = 'work_dirs/rtmpose-s-ear/20230604_234356/vis_data/scalars.json'

In [15]:
with open(log_path, "r") as f:
    json_list = f.readlines()

In [16]:
len(json_list)

62

In [17]:
eval(json_list[4])

{'lr': 0.0008421368421052633,
 'data_time': 7.684620428085327,
 'loss': 0.7676273703575134,
 'loss_kpt': 0.7676273703575134,
 'acc_pose': 0.08557728647014361,
 'time': 8.067522859573364,
 'epoch': 2,
 'iter': 5,
 'memory': 3045,
 'step': 5}

In [18]:
df_train = pd.DataFrame()
df_test = pd.DataFrame()
for each in tqdm(json_list):
    if 'coco/AP' in each:
        df_test = df_test._append(eval(each), ignore_index=True)
    else:
        df_train = df_train._append(eval(each), ignore_index=True)

100%|██████████| 62/62 [00:00<00:00, 3095.35it/s]


In [19]:
df_train

,lr,data_time,loss,loss_kpt,acc_pose,time,epoch,iter,memory,step
0,4.000000e-08,9.073196,0.844231,0.844231,0.009673,10.348924,1.0,1.0,2984.0,1.0
1,2.105642e-04,8.213577,0.844642,0.844642,0.008185,8.933137,1.0,2.0,3045.0,2.0
2,4.210884e-04,7.156120,0.738825,0.738825,0.020238,7.683250,1.0,3.0,2007.0,3.0
3,6.316126e-04,7.531442,0.762052,0.762052,0.045387,7.968434,2.0,4.0,3045.0,4.0
4,8.421368e-04,7.684620,0.767627,0.767627,0.085577,8.067523,2.0,5.0,3045.0,5.0
5,1.052661e-03,7.275690,0.733868,0.733868,0.095421,7.612671,2.0,6.0,2007.0,6.0
6,1.263185e-03,7.518374,0.733210,0.733210,0.170387,7.830858,3.0,7.0,3045.0,7.0
7,1.473709e-03,7.634359,0.730777,0.730777,0.163183,7.928499,3.0,8.0,3045.0,8.0
8,1.684234e-03,7.375135,0.695475,0.695475,0.227381,7.647751,3.0,9.0,2007.0,9.0
9,1.894758e-03,7.490722,0.705664,0.705664,0.163041,7.752536,4.0,10.0,3045.0,10.0


In [20]:
df_test

,coco/AP,coco/AP .5,coco/AP .75,coco/AP (M),coco/AP (L),coco/AR,coco/AR .5,coco/AR .75,coco/AR (M),coco/AR (L),PCK,AUC,NME,data_time,time,step
0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.153061,0.002608,0.324942,0.736847,0.765534,10.0
1,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.297052,0.006859,0.240470,0.662420,0.681080,20.0
